In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import randint
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import pandas as pd
import optuna
import os
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
train= pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')
df= train
train

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

In [ ]:
train

In [ ]:
list= ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
for col in list:
    print(col+":", train[col].unique())

In [ ]:
f = plt.figure(figsize=(25,20))
f.add_subplot(331)
sns.histplot(data= train, x= 'math score');
f.add_subplot(332)
sns.histplot(data= train, x= 'reading score');
f.add_subplot(333)
sns.histplot(data= train, x= 'writing score');

# **Univariate Analysis**

Univariate analysis is the simplest form of analyzing data. “Uni” means “one”, so in other words your data has only one variable. It doesn’t deal with causes or relationships (unlike regression ) and it’s major purpose is to describe; It takes data, summarizes that data and finds patterns in the data.

Checking gender count:

In [ ]:
fig, ax = plt.subplots()
train['gender'].value_counts().plot(ax=ax, kind='bar')

We can see that the number of females is slightly higher than males in this dataset. 

In [ ]:
f = plt.figure(figsize=(25,20))
f.add_subplot(331)
sns.histplot(data= train, x= 'math score', hue= 'gender', multiple= 'stack');
f.add_subplot(332)
sns.histplot(data= train, x= 'reading score', hue= 'gender', multiple= 'stack');
f.add_subplot(333)
sns.histplot(data= train, x= 'writing score', hue= 'gender', multiple= 'stack');

We can deduce that the percentage of females is higher at each test score. 

Now, we'll do the same for the rest of the features: 

In [ ]:
catigorical_features = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']

In [ ]:
for col in catigorical_features:
    print('-'*50)
    print(f'The column name is: {col}')
    print(train[col].value_counts())

In [ ]:
f = plt.figure(figsize=(30,20))
f.add_subplot(331)
sns.countplot(x= 'race/ethnicity', data= train);
f.add_subplot(332)
sns.countplot(x= 'gender', data= train);
f.add_subplot(333)
sns.countplot(x= 'parental level of education', data= train);
f.add_subplot(334)
sns.countplot(x= 'lunch', data= train);
f.add_subplot(336)
sns.countplot(x= 'test preparation course', data= train);

# **Multivariate Analysis**

Multivariate analysis examines two or more variables.

Examinging the count of each gender in each category: 

In [ ]:
f = plt.figure(figsize=(30,20))
f.add_subplot(331)
sns.countplot(data= train,x='race/ethnicity',hue= 'gender');
f.add_subplot(332)
sns.countplot(data= train,x='lunch',hue= 'gender');
f.add_subplot(334)
sns.countplot(data= train,x='test preparation course',hue= 'gender');
f.add_subplot(335)
sns.countplot(data= train,x='parental level of education',hue= 'gender');


Examining the score for each gender in each category of parental level of education: 

In [ ]:
f = plt.figure(figsize=(30,20))
f.add_subplot(331)
sns.barplot(data= train, x='parental level of education',
              y='math score',
              hue= 'gender')
plt.xticks(rotation= 45);
f.add_subplot(332)
sns.barplot(data= train, x='parental level of education',
              y='reading score',
              hue= 'gender')
plt.xticks(rotation= 45);
f.add_subplot(333)
sns.barplot(data= train, x='parental level of education',
              y='writing score',
              hue= 'gender')
plt.xticks(rotation= 45);

Another way to examine the count of each gender in each category of race/ethnicity: 

In [ ]:
numerical_features=['math score', 'reading score', 'writing score']

In [ ]:
pd.crosstab(train['race/ethnicity'], train['gender']).T

Examining the mean score for each test for every group of race/ethnicity: 

In [ ]:
train.groupby(['race/ethnicity'])[numerical_features].agg([np.mean]).T

group E scores the highest compared to students who belong to other groups.

In [ ]:
train.groupby(['test preparation course'])[numerical_features].agg([np.mean]).T

Those who complete test preparation course generally do better. 

In [ ]:
train.groupby(['lunch'])[numerical_features].agg([np.mean]).T

Those who have a standard lunch generally score better.

In [ ]:
train.groupby(['parental level of education'])[numerical_features].agg([np.mean]).T

Students whose parents have a masters degree score higher, but not significantly higher than those whose parents have a bachelor's degree.

Checking for correlations between variables: 

In [ ]:
train.corr()

In [ ]:
corr = train.corr()
f, ax = plt.subplots(figsize=(20, 8))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr,linewidths=.5, annot= True)

Another way to represent correlation between scores: 

In [ ]:
sns.pairplot(train[numerical_features]);

We can dedure that reading and writing scores are more correlated with each other math and reading/writing scores. 

# **Feature Engineering**

Calculate the final percentage of each student using the available three test scores: 

In [ ]:
train['Final Percentage']= ((train['math score']+train['reading score']+train['writing score'])/3)


In [ ]:
train['Final Percentage']

divide the total grade as categories and then use pairplot 
// I am not sure I want do this now, might go first for pass/ fail and then divide into categories. 

Devide the final percentage into bins to determine a reasonable grade distribution:

In [ ]:
train['Final Percentage'].value_counts(bins=10, sort=False)

Get the mean final percentage of students: 

In [ ]:
train['Final Percentage'].mean()

Categorizing students into pass/fail:

In [ ]:
train['Pass_Final'] = (train['Final Percentage'] > 40).map({True: "P",
                                                     False: "F"})

train['Pass_Final'].value_counts()

In [ ]:
sns.countplot(data=train, x= 'Pass_Final');

Assigning a grade to each student based on the final percentage: 

In [ ]:
def getGPA(grade):
    if grade > 80:
        return 'A'
    elif grade> 70:
        return 'B'
    elif grade > 60:
        return 'C'
    elif grade > 50:
        return 'D'
    elif grade > 40:
        return 'E'
    else:
        return 'F' 

In [ ]:
train['GPA'] = train.apply(lambda df : getGPA(df['Final Percentage']), axis= 1)
train.head()      

Grade distribution:

In [ ]:
GPA_Order = ['A','B','C','D','E']
sns.catplot(data= train, x= 'GPA',order= GPA_Order, kind= 'count')
plt.title('Number Of Students/GPA')